In [322]:
import pandas as pd
import numpy as np

from keras import layers
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Flatten, MaxPooling1D, Input, Concatenate
from keras.models import load_model

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [323]:
def construct_pred_matrix(pred):
    arr = []
    for i in pred:
        if i[0] < 0.1:
            arr.append(0)
        else:
            arr.append(1)
    return np.array(arr)   

In [324]:
use_map = True
df = pd.read_csv('./data/spam.csv',delimiter=',')

In [325]:
df = df.sample(frac=1)
X = df['content']
Y = df['category']
map_class1 = {'spam': 0, 'ham':1}
map_class2 = {'positive': 0, 'negative': 1}
if use_map:
    Y = Y.map(map_class1)
Y

4510    1
1984    1
5560    1
4604    1
143     1
       ..
2207    0
3473    1
3994    1
3765    1
4749    1
Name: category, Length: 5572, dtype: int64

In [326]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [327]:
X_train,X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

In [328]:
vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary
training_portion = .8
units = 1024
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [329]:
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
train_padded

array([[ 571, 1445,  880, ...,    0,    0,    0],
       [   2,  483,  605, ...,    0,    0,    0],
       [  94,    3,  208, ...,    0,    0,    0],
       ...,
       [  26, 1363,  263, ...,    0,    0,    0],
       [   2,  166,   81, ...,    0,    0,    0],
       [ 896, 1543,    6, ...,    0,    0,    0]], dtype=int32)

In [334]:
embedding_dim = 128

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen, trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fit model
history = model.fit(train_padded, y_train,
                    epochs=10,
                    verbose=True,
                    batch_size=10)

Epoch 1/10
446/446 [==============================] - 3s 6ms/step - loss: 0.4190 - accuracy: 0.8502
Epoch 2/10
446/446 [==============================] - 3s 6ms/step - loss: 0.1369 - accuracy: 0.9510
Epoch 3/10
446/446 [==============================] - 3s 6ms/step - loss: 0.0605 - accuracy: 0.9850
Epoch 4/10
446/446 [==============================] - 3s 7ms/step - loss: 0.0357 - accuracy: 0.9918
Epoch 5/10
446/446 [==============================] - 3s 7ms/step - loss: 0.0194 - accuracy: 0.9955
Epoch 6/10
446/446 [==============================] - 3s 7ms/step - loss: 0.0112 - accuracy: 0.9993
Epoch 7/10
446/446 [==============================] - 3s 7ms/step - loss: 0.0067 - accuracy: 0.9993
Epoch 8/10
446/446 [==============================] - 3s 7ms/step - loss: 0.0046 - accuracy: 0.9990
Epoch 9/10
446/446 [==============================] - 3s 7ms/step - loss: 0.0049 - accuracy: 0.9988
Epoch 10/10
446/446 [==============================] - 3s 7ms/step - loss: 0.0022 - accuracy: 0.9996

In [335]:
validation_sequences = tokenizer.texts_to_sequences(X_test)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [336]:
y_train
    

4629    0
2731    1
1978    0
3336    1
2331    1
       ..
3673    1
4977    1
3143    1
3593    1
3169    1
Name: category, Length: 4457, dtype: int64

In [337]:
pred = model.predict(validation_padded)  
pred_matrix = construct_pred_matrix(pred)

In [338]:
from sklearn import preprocessing, linear_model, naive_bayes, metrics, svm
metrics.accuracy_score(pred_matrix, y_test)

0.9802690582959641